#### Credits to Avi Chawla.
#### I am modifying his notebook found on his Google Colab to try running it on my personal computer to time the difference.
#### https://www.blog.dailydoseofds.com/p/nvidias-latest-update-can-make-your
#### https://colab.research.google.com/drive/1l6mfdEFehLnxRMzGXenI9N3CBqWh65ve?usp=sharing#scrollTo=ArOr3GrJTDMq

## Download the Data to Process

In [1]:
# This command will download the file to the home directory
!wget https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet

--2023-11-15 11:41:03--  https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet
Resolving data.rapids.ai (data.rapids.ai)... 13.224.250.77, 13.224.250.61, 13.224.250.33, ...
Connecting to data.rapids.ai (data.rapids.ai)|13.224.250.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 477161608 (455M) [binary/octet-stream]
Saving to: ‘nyc_parking_violations_2022.parquet’

nyc_parking_violati 100%[===================>] 455.06M  58.0MB/s    in 7.7s    

2023-11-15 11:41:12 (58.8 MB/s) - ‘nyc_parking_violations_2022.parquet’ saved [477161608/477161608]



## WITHOUT RAPIDS.ai CUDF

In [2]:
# Note the location of pandas
import pandas as pd
pd

<module 'pandas' from '/home/lsz/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/pandas/__init__.py'>

In [3]:
%%time
df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description",
             "Vehicle Body Type", "Issue Date", "Summons Number"]
)

CPU times: user 4.99 s, sys: 16.8 s, total: 21.8 s
Wall time: 16.7 s


In [4]:
%%time
(df[["Registration State", "Violation Description"]]
 .value_counts()
 .groupby("Registration State")
 .head(1)
 .sort_index()
 .reset_index()
)

CPU times: user 2.38 s, sys: 1.25 s, total: 3.63 s
Wall time: 3.58 s


,Registration State,Violation Description,0
0,99,74-Missing Display Plate,835
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
60,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
61,WA,21-No Parking (street clean),3732
62,WI,14-No Standing,1639
63,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


In [5]:
%%time
(df
 .groupby(["Vehicle Body Type"])
 .agg({"Summons Number": "count"})
 .rename(columns={"Summons Number": "Count"})
 .sort_values(["Count"], ascending=False)
)

CPU times: user 1.48 s, sys: 30 ms, total: 1.51 s
Wall time: 1.5 s


,Count
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
CARY,1
ISUZ,1
IXMR,1


In [6]:
%%time
weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

CPU times: user 2.49 s, sys: 219 ms, total: 2.71 s
Wall time: 2.71 s


issue_weekday
Sunday        462992
Saturday     1108385
Monday       2488563
Wednesday    2760088
Tuesday      2809949
Friday       2891679
Thursday     2913951
Name: Summons Number, dtype: int64

## WITH RAPIDS.ai CUDF

In [7]:
# Even after we load_ext, pd is still linked to before and will not have any changes.
%load_ext cudf.pandas
pd

<module 'pandas' from '/home/lsz/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/pandas/__init__.py'>

In [8]:
# We need to re-import pandas as pd AFTER the %load_ext cudf.pandas
# Can see that the location has changed.
import pandas as pd
pd

<module 'pandas' (ModuleAccelerator(fast=cudf, slow=pandas))>

In [9]:
%%time
df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description",
             "Vehicle Body Type", "Issue Date", "Summons Number"]
)

CPU times: user 1.06 s, sys: 602 ms, total: 1.66 s
Wall time: 2.1 s


In [10]:
%%time
(df[["Registration State", "Violation Description"]]
 .value_counts()
 .groupby("Registration State")
 .head(1)
 .sort_index()
 .reset_index()
)

CPU times: user 71 ms, sys: 0 ns, total: 71 ms
Wall time: 84.9 ms


,Registration State,Violation Description,0
0,99,74-Missing Display Plate,835
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
60,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
61,WA,21-No Parking (street clean),3732
62,WI,14-No Standing,1639
63,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


In [11]:
%%time
(df
 .groupby(["Vehicle Body Type"])
 .agg({"Summons Number": "count"})
 .rename(columns={"Summons Number": "Count"})
 .sort_values(["Count"], ascending=False)
)

CPU times: user 25.1 ms, sys: 7.01 ms, total: 32.1 ms
Wall time: 37.7 ms


,Count
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
YANT,1
YBSD,1
YEL,1


In [12]:
%%time
weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

CPU times: user 165 ms, sys: 1.47 ms, total: 166 ms
Wall time: 198 ms


issue_weekday
Sunday        462992
Saturday     1108385
Monday       2488563
Wednesday    2760088
Tuesday      2809949
Friday       2891679
Thursday     2913951
Name: Summons Number, dtype: int64